In [2]:
import pandas as pd 
import numpy as np
import statsmodels.api as sm

/home/creambbq/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:7: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import (to_datetime, Int64Index, DatetimeIndex, Period,
/home/creambbq/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:7: FutureWarning: pandas.Float64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import (to_datetime, Int64Index, DatetimeIndex, Period,


In [30]:
df = pd.read_stata('cornwell.dta')
df = df.assign(cte = 1)
df = df[['county', 'year', 'lcrmrte','lprbarr', 'lprbconv', 'lprbpris', 'lavgsen', 'lpolpc']].assign(cte = 1)
df = df.dropna()
df = pd.concat([df, pd.get_dummies(df['year'])], axis = 1 )
x = df[['cte', 'lprbarr', 'lprbconv', 'lprbpris', 'lavgsen', 'lpolpc']]
x = pd.concat([x, pd.get_dummies(df['year'])], axis = 1 )
y = df['lcrmrte']

OLS COMÚN

In [ ]:
pols = sm.OLS(y,x).fit()
pols.summary()


Errores estandar robustos a la presencia de heterocedasticidad

In [ ]:
pols_rob = sm.OLS(y,x).fit(cov_type='HC3')
pols_rob.summary()

## Ejercicio Número 3

In [32]:
np.random.seed(1313)
n = 5 
T = 2
omg = np.array([[1,0], [0,4]])
muestras = 1000
data = {'i': [], 't': [], 'y': [], 'cte':[] , 'x': [], 'u': []}


for i in range(1,n+1) : 
    for t in range(1,T+1) : 
        data['cte'].append(1)
        data['i'].append(i)
        data['t'].append(t)
        x = np.random.uniform(1,20,1)
        x = x.astype(float)
        u = np.random.normal(0,omg.item(t-1,t-1))
        data['x'].append(x)
        data['u'].append(u)
        data['y'].append(1 + x + u)


data = pd.DataFrame(data=data)
y = data['y'].astype(float)
x = data[['cte', 'x']].astype(float)

In [ ]:
reg_ols = sm.OLS(y,x).fit()

In [67]:
u_hat = {'i': data['i'], 't': data['t'], 'u_jt': reg_ols.resid.tolist()}
u_hat = pd.DataFrame(data = u_hat)
u_hat

,i,t,u_jt
0,1,1,-2.619755
1,1,2,-0.951289
2,2,1,-0.427754
3,2,2,-0.389934
4,3,1,-0.548701
5,3,2,1.604623
6,4,1,-0.465574
7,4,2,4.475627
8,5,1,-0.132044
9,5,2,-0.545199


In [72]:
omg_est = np.array([[0,0], [0,0]])
for l in range(0,n*T-1): 
    u_j = np.array([[u_hat['u_jt'][l]], [u_hat['u_jt'][l+1]]])
    omg_est = omg_est + u_j.dot(u_j.T)
omg_est

array([[31.24440301, -0.95044039],
       [-0.95044039, 24.67852892]])

In [71]:
res = np.array([[0,0], [0,0]])
for h in range(0,4): 
    matriz = np.array([[1,1], [1,1]])
    res = res + matriz

res

array([[4, 4],
       [4, 4]])